In [1]:
%pip install beautifulsoup4 requests pandas -q

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [112]:
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

In [4]:
df = pd.read_csv('AllAnimes.csv')
df.sample(5)

,position,title,episodes,release_date,members,score,details
7014,7016.0,Classroom☆Crisis: Tabi no Haji wa Uwanuri,Special (1 eps),Dec 2015 - Dec 2015,6229,6.40,https://myanimelist.net/anime/31530/Classroom☆...
2189,2191.0,Hikaru no Go: Sabaki no Ikkyoku! Inishie no Ha...,OVA (1 eps),2002 - 2002,6623,7.37,https://myanimelist.net/anime/10765/Hikaru_no_...
23014,NaN,Sei Michaela Gakuen Hyouryuuki II,OVA (2 eps),Oct 1994 - Nov 1995,1187,4.60,https://myanimelist.net/anime/2240/Sei_Michael...
18033,18043.0,Dochamon Junior 2,TV (16 eps),Oct 2016 - Feb 2017,245,0.00,https://myanimelist.net/anime/34207/Dochamon_J...
1769,1771.0,Saint☆Oniisan,OVA (2 eps),Dec 2012 - Aug 2013,88736,7.48,https://myanimelist.net/anime/15775/Saint☆Oniisan


In [113]:
def get_details(details_page):
    details = [np.nan for i in range(7)]
    for i in details_page.find_all('div', {'class': 'spaceit_pad'}):
        if 'Broadcast:' in i.text:
            details[0] = i.text.split()[1:]
            pass
        if 'Studios:' in i.text:
            details[1] = i.find('a').text
            pass
        if 'Source:' in i.text:
            details[2] = i.text.split()[1].strip()
            pass
        if 'Genres:' in i.text:
            details[3] = [r.text for r in i.find_all('span')][1:]
            pass
        if 'Theme:' in i.text or 'Themes:' in i.text:
            details[4] = [r.text for r in i.find_all('span')][1:]
            pass
        if 'Demographic:' in i.text:
            details[5] = i.find('a').text
            pass
        if 'Popularity:' in i.text:
            details[6] = i.text.split('#')[1].strip()
            pass
    return details

In [118]:
details = []
for i in range(len(df)):
    res = requests.get(df.details[i])
    soup = BeautifulSoup(res.text, 'html.parser')
    details_page = soup.find('div', {'class': 'leftside'})
    details.append(get_details(details_page))
    print(f'Get details for {df.title[i]}, {i+1}/{len(df)}')

Get details for Fullmetal Alchemist: Brotherhood, 1/23070
Get details for Kaguya-sama wa Kokurasetai: Ultra Romantic, 2/23070
Get details for Bleach: Sennen Kessen-hen, 3/23070
Get details for Gintama°, 4/23070
Get details for Steins;Gate, 5/23070
Get details for Shingeki no Kyojin Season 3 Part 2, 6/23070
Get details for Gintama', 7/23070
Get details for Gintama: The Final, 8/23070
Get details for Gintama': Enchousen, 9/23070
Get details for Hunter x Hunter (2011), 10/23070
Get details for Ginga Eiyuu Densetsu, 11/23070
Get details for Fruits Basket: The Final, 12/23070
Get details for Gintama., 13/23070
Get details for 3-gatsu no Lion 2nd Season, 14/23070
Get details for Koe no Katachi, 15/23070
Get details for Clannad: After Story, 16/23070
Get details for Gintama, 17/23070
Get details for Violet Evergarden Movie, 18/23070
Get details for Gintama Movie 2: Kanketsu-hen - Yorozuya yo Eien Nare, 19/23070
Get details for Code Geass: Hangyaku no Lelouch R2, 20/23070
Get details for Chain

In [119]:
df_details = pd.DataFrame(details, columns=['broadcast', 'studios', 'source','genres', 'themes', 'demographic', 'popularity'])
df_details

,broadcast,studios,source,genres,themes,demographic,popularity
0,"[Sundays, at, 17:00, (JST)]",Bones,Manga,"[Action, Adventure, Drama, Fantasy]",[Military],Shounen,3
1,"[Saturdays, at, 00:00, (JST)]",A-1 Pictures,Manga,"[Comedy, Romance]","[Psychological, School]",Seinen,242
2,"[Tuesdays, at, 00:00, (JST)]",Pierrot,Manga,"[Action, Adventure, Fantasy]",NaN,Shounen,748
3,"[Wednesdays, at, 18:00, (JST)]",Bandai Namco Pictures,Manga,"[Action, Comedy, Sci-Fi]","[Gag Humor, Historical, Parody, Samurai]",Shounen,337
4,"[Wednesdays, at, 02:05, (JST)]",White Fox,Visual,"[Drama, Sci-Fi, Suspense]","[Psychological, Time Travel]",NaN,13
...,...,...,...,...,...,...,...
23065,NaN,Studio Kyuuma,Visual,NaN,NaN,NaN,15105
23066,NaN,Shion,Manga,NaN,NaN,NaN,14519
23067,NaN,add some,Original,"[Avant Garde, Drama, Erotica]",NaN,NaN,19681
23068,NaN,add some,Original,NaN,NaN,NaN,19474


In [120]:
_df = df[:][:len(df_details)]
_df = pd.concat([_df, df_details], axis=1)
_df.sample()

,position,title,episodes,release_date,members,score,details,broadcast,studios,source,genres,themes,demographic,popularity
20899,20910.0,Yukueshirezu,Music (1 eps),Oct 2022 - Oct 2022,106,0.0,https://myanimelist.net/anime/53336/Yukueshirezu,NaN,add some,Original,NaN,"[Music, Psychological]",NaN,19326


In [121]:
_df.to_csv('AllAnimesFull.csv', index=False)